In [1]:
%reload_kedro

[08/17/24 11:57:46] INFO     Resolved project path as: /Users/sharkovgrigory/code/fraud_detection.  ]8;id=508244;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=448632;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py#171\171]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

                    INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=842697;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=453793;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py#65\65]8;;\

                    WARNING  No 'mlflow.yml' config file found in environment. Default            ]8;id=363074;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=344494;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py#78\78]8;;\
                             configuration will be used. Use ``kedro mlflow init`` command in CLI                  
                             to customize the configuration.                                                       

                    INFO     The 'tracking_uri' key in mlflow.yml is relative            ]8;id=770819;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/config/kedro_mlflow_config.py\kedro_mlflow_config.py]8;;\:]8;id=627878;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/config/kedro_mlflow_config.py#260\260]8;;\
                             ('server.mlflow_(tracking|registry)_uri = mlruns'). It is                             
                             converted to a valid uri:                                                             
                             'file:///Users/sharkovgrigory/code/fraud_detection/mlruns'                            

                    INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=560558;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=696880;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

                    WARNING  Failed to send data to Heap. Exception of type 'ConnectionError' was     ]8;id=19010;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=410801;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_telemetry/plugin.py#342\342]8;;\
                             raised.                                                                               

                    INFO     Kedro project fraud_detection                                          ]8;id=488570;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=212730;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py#141\141]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=467290;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=158612;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py#142\142]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=107958;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=766227;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py#148\148]8;;\

In [2]:
from fraud_detection.utils.row_preprocessor import get_row_preprocessor, FAST_MODEL_SETTINGS

In [3]:
train_data = catalog.load("combined_train_data")
id_cols = context.params.get("train_set_parameters").get("columns").get('id_columns')
cat_cols = ["merchant", "category"] #, "gender"]
num_cols = ["amt", 'city_pop']

train_data.head()

                    INFO     Loading data from combined_train_data (CSVDataset)...              ]8;id=721583;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=325608;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/io/data_catalog.py#508\508]8;;\

,cc_num,window_id,label,merchant,category,amt,gender,city,state,city_pop,job
0,60416207185,60416207185_115,True,"fraud_Schroeder, Hauck and Treutel",entertainment,90.60,F,Fort Washakie,WY,1645,Information systems manager
1,60416207185,60416207185_115,True,"fraud_Hills, Hegmann and Schaefer",health_fitness,26.13,F,Fort Washakie,WY,1645,Information systems manager
2,60416207185,60416207185_115,True,"fraud_Langworth, Boehm and Gulgowski",shopping_net,1.24,F,Fort Washakie,WY,1645,Information systems manager
3,60416207185,60416207185_115,True,fraud_Turcotte-Halvorson,misc_pos,11.60,F,Fort Washakie,WY,1645,Information systems manager
4,60416207185,60416207185_115,True,fraud_Padberg-Sauer,home,200.83,F,Fort Washakie,WY,1645,Information systems manager


In [4]:
preprocessor = get_row_preprocessor(cat_cols, num_cols, FAST_MODEL_SETTINGS)
preprocessor.fit(train_data)

[08/17/24 11:57:47] WARNING  Each 'sentences' item should be a list of words (usually unicode       ]8;id=982359;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/gensim/models/word2vec.py\word2vec.py]8;;\:]8;id=740496;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/gensim/models/word2vec.py#558\558]8;;\
                             strings). First item here is instead plain <class 'str'>.                             

[FeatureUnion]  (step 1 of 4) Processing encoder_pipeline_merchant, total=  11.0s


[08/17/24 11:57:58] WARNING  Each 'sentences' item should be a list of words (usually unicode       ]8;id=565114;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/gensim/models/word2vec.py\word2vec.py]8;;\:]8;id=861400;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/gensim/models/word2vec.py#558\558]8;;\
                             strings). First item here is instead plain <class 'str'>.                             

[FeatureUnion]  (step 2 of 4) Processing encoder_pipeline_category, total=   7.9s
[FeatureUnion]  (step 3 of 4) Processing numeric_pipeline_amt, total=   0.0s
[FeatureUnion]  (step 4 of 4) Processing numeric_pipeline_city_pop, total=   0.0s


Pipeline(steps=[('combined_embeddings',
                 FeatureUnion(n_jobs=1,
                              transformer_list=[('encoder_pipeline_merchant',
                                                 Pipeline(steps=[('encoder_merchant',
                                                                  FastTextEncoder(column='merchant',
                                                                                  embedder_settings={'min_count': 3,
                                                                                                     'vector_size': 3,
                                                                                                     'window': 5,
                                                                                                     'workers': 4}))])),
                                                ('encoder_pipeline_category',
                                                 Pipeline(steps=[('encoder_category',
                                                                  FastTextEncoder(column='categ...
                                                                  StandardScaler()),
                                                                 ('imputer_amt',
                                                                  SimpleImputer())])),
                                                ('numeric_pipeline_city_pop',
                                                 Pipeline(steps=[('selector_city_pop',
                                                                  FunctionTransformer(func=<function get_row_preprocessor.<locals>.<listcomp>.<lambda> at 0x177725d00>)),
                                                                 ('scaler_city_pop',
                                                                  StandardScaler()),
                                                                 ('imputer_city_pop',
                                                                  SimpleImputer())]))],
                              verbose=True)),
                ('concatenate_features', ConcatenateTransformer())])

In [5]:
preprocessor.transform(train_data.loc[:1,:])


[
    [
        -0.014516915,
        0.0045382995,
        0.014889569,
        0.017116437,
        -0.082998045,
        0.0573985,
        0.060293645,
        0.078404866,
        0.019872919,
        -0.04159637,
        0.043534033,
        -0.0540257,
        -0.002666257,
        0.024426937,
        -0.033753067,
        -0.29084700339951247,
        -0.29084700339951247
    ],
    [
        -0.08941088,
        0.059118245,
        0.04565257,
        -0.030436808,
        -0.015588927,
        -0.07261914,
        0.060293645,
        0.078404866,
        0.019872919,
        0.011420587,
        -0.041980065,
        0.07569261,
        0.0070926007,
        0.005174372,
        -0.017350368,
        -0.29084700339951247,
        -0.29084700339951247
    ]
]

In [7]:
preprocessor.transform(train_data.loc[:1,:])[0]

[08/15/24 11:35:35] WARNING  /Users/sharkovgrigory/code/fraud_detection/src/fraud_detection/utils/c ]8;id=839412;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=125371;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/warnings.py#109\109]8;;\
                             oncatenate_transformer.py:18: VisibleDeprecationWarning: Creating an                  
                             ndarray from ragged nested sequences (which is a list-or-tuple of                     
                             lists-or-tuples-or ndarrays with different lengths or shapes) is                      
                             deprecated. If you meant to do this, you must specify 'dtype=object'                  
                             when creating the ndarray.                                                            
                               return np.array(return_list)                                                        
                                                                                                                   

array([-0.01451692,  0.0045383 ,  0.01488957], dtype=float32)